# Importing libraries

In [1]:
import pandas as pd
import joblib
import time

import numpy as np
from numpy import nan
import geopandas as gpd
from shapely.geometry import Point, box

from collections import defaultdict
from itertools import cycle

from sklearn.metrics import accuracy_score, roc_curve, auc, roc_auc_score, classification_report
from sklearn.preprocessing import label_binarize, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import torch
import torch.nn as nn

import matplotlib.pyplot as plt



import os
os.chdir(r"E:\Data challenge")

# Importing Data

In [2]:
#data_drias = pd.read_csv(r"data\Drias_data\RCP_4.5.csv", sep=";", header=31)
data_drias = pd.read_csv(r"data\Drias_data\RCP_4.5_with_distance.csv", sep=";")

In [3]:
data_Flood = pd.read_csv(r"data\Flood\flood_risk_results.csv")

In [4]:
data_clay = gpd.read_file(r"data\Flood\clay_risk_results.csv")
data_clay = data_clay.astype({'longitude': 'Float64', 'latitude': 'Float64'})

# Preparing Data Flood

In [5]:
data_clay["alea"] = data_clay["alea"].replace({"Faible": 1,"Moyen": 2,"Fort": 3})
data_clay.drop(columns=['niveau'],inplace=True)
data_clay["alea"] = data_clay["alea"].replace("", pd.NA)
data_clay["alea"] = data_clay["alea"].fillna(0)
data_clay = data_clay.astype({'alea': 'int32'})

C:\Users\Lucas\AppData\Local\Temp\ipykernel_8960\2279142045.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_clay["alea"] = data_clay["alea"].fillna(0)


In [6]:
data_Flood_High = data_Flood.copy()
data_Flood_High["scenario"] = data_Flood_High["scenario"].apply(lambda x: "High" if "high" in x else None)
data_Flood_High["ht"] = data_Flood_High["ht"].apply(lambda x: eval(x)['high'])
data_Flood_High["ht"] = data_Flood_High["ht"].apply(lambda x: max(set([tuple(l) for l in x]), key=x.count, default=None))

data_Flood_High["ht_min"] = data_Flood_High["ht"].str[0]
data_Flood_High["ht_max"] = data_Flood_High["ht"].str[-1]
data_Flood_High.drop(columns=["ht"], inplace=True)

data_Flood_High.loc[data_Flood_High['scenario'].notnull()]

,Unnamed: 0,longitude,latitude,scenario,ht_min,ht_max
47,47,8.6413,41.9452,High,0.5,1.0
48,48,8.7372,41.9394,High,2.0,4.0
49,49,8.8330,41.9335,High,2.0,4.0
58,58,8.8409,42.0051,High,0.5,1.0
160,160,2.8711,42.5634,High,1.0,2.0
...,...,...,...,...,...,...
26908,26908,3.0718,50.7606,High,2.0,9999.0
26915,26915,2.1665,50.8345,High,0.0,0.0
26930,26930,1.8252,50.9770,High,2.0,9999.0
26931,26931,1.9388,50.9774,High,0.5,1.0


In [7]:
data_Flood_mid = data_Flood.copy()
data_Flood_mid["scenario"] = data_Flood_mid["scenario"].apply(lambda x: "Mid" if "mid" in x else None)
data_Flood_mid["ht"] = data_Flood_mid["ht"].apply(lambda x: eval(x)['mid'])
data_Flood_mid["ht"] = data_Flood_mid["ht"].apply(lambda x: max(set([tuple(l) for l in x]), key=x.count, default=None))

data_Flood_mid["ht_min"] = data_Flood_mid["ht"].str[0]
data_Flood_mid["ht_max"] = data_Flood_mid["ht"].str[-1]
data_Flood_mid.drop(columns=["ht"], inplace=True)

data_Flood_mid.loc[data_Flood_mid['scenario'].notnull()]

,Unnamed: 0,longitude,latitude,scenario,ht_min,ht_max
47,47,8.6413,41.9452,Mid,0.5,1.0
48,48,8.7372,41.9394,Mid,0.5,1.0
49,49,8.8330,41.9335,Mid,0.5,1.0
58,58,8.8409,42.0051,Mid,0.5,1.0
160,160,2.8711,42.5634,Mid,1.0,2.0
...,...,...,...,...,...,...
26908,26908,3.0718,50.7606,Mid,2.0,9999.0
26922,26922,1.8260,50.9053,Mid,0.0,0.5
26930,26930,1.8252,50.9770,Mid,2.0,9999.0
26931,26931,1.9388,50.9774,Mid,0.5,1.0


In [8]:
data_Flood_low = data_Flood.copy()
data_Flood_low["scenario"] = data_Flood_low["scenario"].apply(lambda x: "Low" if "low" in x else None)
data_Flood_low["ht"] = data_Flood_low["ht"].apply(lambda x: eval(x)['low'])
data_Flood_low["ht"] = data_Flood_low["ht"].apply(lambda x: max(set([tuple(l) for l in x]), key=x.count, default=None))

data_Flood_low["ht_min"] = data_Flood_low["ht"].str[0]
data_Flood_low["ht_max"] = data_Flood_low["ht"].str[-1]
data_Flood_low.drop(columns=["ht"], inplace=True)

data_Flood_low.loc[data_Flood_low['scenario'].notnull()]

,Unnamed: 0,longitude,latitude,scenario,ht_min,ht_max
47,47,8.6413,41.9452,Low,0.0,1.0
48,48,8.7372,41.9394,Low,0.0,1.0
49,49,8.8330,41.9335,Low,0.0,1.0
58,58,8.8409,42.0051,Low,0.5,1.0
160,160,2.8711,42.5634,Low,1.0,2.0
...,...,...,...,...,...,...
26890,26890,2.8448,50.6900,Low,0.5,1.0
26892,26892,3.0707,50.6889,Low,2.0,9999.0
26901,26901,2.2799,50.7628,Low,2.0,999.0
26905,26905,2.7324,50.7622,Low,0.5,1.0


# Join our different Dataset

### concatenate 3 intermediate Flood risk dataset


In [9]:
Flood_list = [data_Flood_High,data_Flood_mid,data_Flood_low]
Data_Flood = pd.concat(Flood_list)


Join **Flood risk** data with **Drias** data

In [10]:
data_merged = Data_Flood.merge(
    data_drias[data_drias['Période']=='H1'],
    left_on=["latitude", "longitude"],
    right_on=["Latitude", "Longitude"],
    how="inner"   # ou "left" selon ton besoin
)


Join our **data_merged** dataset with our datas on **clay soils**.

In [11]:
gdf1 = gpd.GeoDataFrame(data_merged, geometry=gpd.points_from_xy(data_merged.longitude, data_merged.latitude), crs="EPSG:4326")
gdf2 = gpd.GeoDataFrame(data_clay, geometry=gpd.points_from_xy(data_clay.longitude, data_clay.latitude), crs="EPSG:4326")

gdf1 = gdf1.to_crs(3857)
gdf2 = gdf2.to_crs(3857)

data_merged = gpd.sjoin_nearest(gdf1,gdf2.drop(columns=['latitude','longitude']),max_distance=50,distance_col="distance_m")

**Remove unecessary columns** and **clean Nan** and **categorical** datas

In [12]:
#data_merged.drop(columns=['Unnamed: 0','Unnamed: 17','Point','longitude','latitude','ht_min','ht_max','Longitude','Latitude','Contexte','Période','field_1','distance_m','index_right','geometry'], inplace=True)
data_merged.drop(columns=['Unnamed: 0','Unnamed: 17','Point','longitude','latitude','Longitude','Latitude','Contexte','Période','field_1','distance_m','index_right','geometry'], inplace=True)
data_merged["scenario"] = data_merged["scenario"].replace({
    "Low": 3,
    "Mid": 2,
    "High": 1
})
data_merged["ht_min"] = data_merged["ht_min"].fillna(0)
data_merged["ht_max"] = data_merged["ht_max"].fillna(0)
data_merged["scenario"] = data_merged["scenario"].fillna(0)

C:\Users\Lucas\AppData\Local\Temp\ipykernel_8960\828002797.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_merged["scenario"] = data_merged["scenario"].replace({


# MODEL

Our goal is to predict **ht_min** and **ht_max** using our **Drias** dataset, focusing on **clay soils** and the **Flood risk class**.\
For this purpose, we chose to use a **classification** approach (since both variables have a limited number of classes, 6 for ht_min and 10 for ht_max).\
We applied **weights** to account for the **imbalance** in **class distributions**.

### Prepare data

In [13]:
df = data_merged.copy()
df['ht_max'] = df['ht_max'].apply(lambda x: x if x <= 6 else 1000)


y = df[["ht_min","ht_max"]]
X = df.drop(columns=["ht_min","ht_max"])

# garder seulement numériques
X = X.select_dtypes(include=["int32","int64","float64"])


In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train[['NORPAV', 'NORRR', 'NORRR1MM', 'NORPN20MM', 'NORPFL90',
       'NORPXCDD', 'NORPINT', 'NORPQ90', 'NORPQ99', 'NORRR99', 'NORHUSAV',
       'NORETPC','dist_fleuve_km','dist_riviere_km','dist_cote_km']] = scaler.fit_transform(X_train[['NORPAV', 'NORRR', 'NORRR1MM', 'NORPN20MM', 'NORPFL90',
       'NORPXCDD', 'NORPINT', 'NORPQ90', 'NORPQ99', 'NORRR99', 'NORHUSAV',
       'NORETPC','dist_fleuve_km','dist_riviere_km','dist_cote_km']])

X_test[['NORPAV', 'NORRR', 'NORRR1MM', 'NORPN20MM', 'NORPFL90',
       'NORPXCDD', 'NORPINT', 'NORPQ90', 'NORPQ99', 'NORRR99', 'NORHUSAV',
       'NORETPC','dist_fleuve_km','dist_riviere_km','dist_cote_km']] = scaler.transform(X_test[['NORPAV', 'NORRR', 'NORRR1MM', 'NORPN20MM', 'NORPFL90',
       'NORPXCDD', 'NORPINT', 'NORPQ90', 'NORPQ99', 'NORRR99', 'NORHUSAV',
       'NORETPC','dist_fleuve_km','dist_riviere_km','dist_cote_km']])



In [16]:
# Convert to torch tensors
X_train = torch.tensor(X_train.to_numpy(), dtype=torch.float32)
X_test = torch.tensor(X_test.to_numpy(), dtype=torch.float32)

y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_test  = torch.tensor(y_test.values, dtype=torch.float32)


In [17]:
y_train = y_train.to(torch.int64)
y_test  = y_test.to(torch.int64)


num_classes_min = y_train[:,0].max().item() + 1  # assuming classes start at 0
num_classes_max = y_train[:,1].max().item() + 1


In [24]:
def get_class_weights(y_col):
    classes, counts = torch.unique(y_col, return_counts=True)
    weights = counts.float().sum() / (len(classes) * counts.float())
    weight_tensor = torch.zeros(y_col.max()+1)
    weight_tensor[classes] = weights
    return weight_tensor

weight_min = get_class_weights(y_train[:,0])
weight_max = get_class_weights(y_train[:,1])

# -------------------------------
# 3. Define the model
# -------------------------------
class DenseNetClassifier(nn.Module):
    def __init__(self, input_dim, num_classes_min, num_classes_max):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
        )
        self.ht_min = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, num_classes_min)
        )
        self.ht_max = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, num_classes_max)
        )

    def forward(self, x):
        #base = self.net(x)
        return self.ht_min(x), self.ht_max(x)

In [25]:
model = DenseNetClassifier(X_train.shape[1], num_classes_min, num_classes_max)

# -------------------------------
# 4. Loss and optimizer
# -------------------------------
criterion_min = nn.CrossEntropyLoss(weight=weight_min)
criterion_max = nn.CrossEntropyLoss(weight=weight_max)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

# -------------------------------
# 5. Training loop
# -------------------------------
epochs = 500
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    ht_min_logits, ht_max_logits = model(X_train)
    loss_min = criterion_min(ht_min_logits, y_train[:,0])
    loss_max = criterion_max(ht_max_logits, y_train[:,1])
    train_loss = loss_min + loss_max

    train_loss.backward()
    optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_min_logits, val_max_logits = model(X_test)
        val_loss = criterion_min(val_min_logits, y_test[:,0]) + \
                   criterion_max(val_max_logits, y_test[:,1])

        # Accuracy and macro-F1
        val_min_pred = torch.argmax(val_min_logits, dim=1)
        val_max_pred = torch.argmax(val_max_logits, dim=1)
        acc_min = accuracy_score(y_test[:,0].numpy(), val_min_pred.numpy())
        acc_max = accuracy_score(y_test[:,1].numpy(), val_max_pred.numpy())

    if epoch % 20 == 0:
        print(f"Epoch {epoch+1:04d} | "
              f"Train Loss: {train_loss.item():.4f} | "
              f"Val Loss: {val_loss.item():.4f} | ")

Epoch 0001 | Train Loss: 8.6349 | Val Loss: 8.4234 | 
Epoch 0021 | Train Loss: 2.5995 | Val Loss: 2.4514 | 
Epoch 0041 | Train Loss: 1.6632 | Val Loss: 1.5965 | 
Epoch 0061 | Train Loss: 1.3390 | Val Loss: 1.3016 | 
Epoch 0081 | Train Loss: 1.1412 | Val Loss: 1.1034 | 
Epoch 0101 | Train Loss: 0.9778 | Val Loss: 0.9514 | 
Epoch 0121 | Train Loss: 0.8506 | Val Loss: 0.8294 | 
Epoch 0141 | Train Loss: 0.7575 | Val Loss: 0.7381 | 
Epoch 0161 | Train Loss: 0.6912 | Val Loss: 0.6720 | 
Epoch 0181 | Train Loss: 0.6381 | Val Loss: 0.6220 | 
Epoch 0201 | Train Loss: 0.6022 | Val Loss: 0.5880 | 
Epoch 0221 | Train Loss: 0.5713 | Val Loss: 0.5611 | 
Epoch 0241 | Train Loss: 0.5497 | Val Loss: 0.5409 | 
Epoch 0261 | Train Loss: 0.5242 | Val Loss: 0.5234 | 
Epoch 0281 | Train Loss: 0.5058 | Val Loss: 0.5045 | 
Epoch 0301 | Train Loss: 0.4883 | Val Loss: 0.4879 | 
Epoch 0321 | Train Loss: 0.4688 | Val Loss: 0.4730 | 
Epoch 0341 | Train Loss: 0.4529 | Val Loss: 0.4599 | 
Epoch 0361 | Train Loss: 0.4

In [26]:
# Inference
model.eval()
with torch.no_grad():
    ht_min_logits, ht_max_logits = model(X_test)
    ht_min_pred = torch.argmax(ht_min_logits, dim=1)
    ht_max_pred = torch.argmax(ht_max_logits, dim=1)

y_pred_df = pd.DataFrame({
    'ht_min': ht_min_pred.numpy(),
    'ht_max': ht_max_pred.numpy()
})


In [27]:
y_test_df = pd.DataFrame(y_test.numpy(), columns=['ht_min', 'ht_max'])

print("Accuracy ht_min:", accuracy_score(y_test_df['ht_min'], y_pred_df['ht_min']))
print("Accuracy ht_max:", accuracy_score(y_test_df['ht_max'], y_pred_df['ht_max']))

print("\nClassification report ht_min:\n", 
      classification_report(y_test_df['ht_min'], y_pred_df['ht_min']))
print("\nClassification report ht_max:\n", 
      classification_report(y_test_df['ht_max'], y_pred_df['ht_max']))

Accuracy ht_min: 0.9668302472954211
Accuracy ht_max: 0.9763839058655339

Classification report ht_min:
               precision    recall  f1-score   support

           0       1.00      0.97      0.98     70657
           1       0.38      0.96      0.55       337
           2       0.48      0.98      0.64      1747
           4       0.43      1.00      0.60         6

    accuracy                           0.97     72747
   macro avg       0.57      0.98      0.69     72747
weighted avg       0.98      0.97      0.97     72747


Classification report ht_max:
               precision    recall  f1-score   support

           0       1.00      0.99      1.00     68389
           1       0.76      0.64      0.69      2340
           2       0.35      0.91      0.50       284
           3       0.52      0.97      0.68       265
           4       0.45      1.00      0.62        27
           5       1.00      1.00      1.00         3
        1000       0.68      0.70      0.69      1

Our **precision** is decreasing with the number of occurences in the class but our **recall** remains high enough which means that even if our model predict false positive for classes that appear rarely, it still manage to predict true positive.

In [28]:
y_pred_df.value_counts()

ht_min  ht_max
0       0         67976
2       1000       1444
        1          1343
        3           450
1       2           414
        1           389
2       2           317
0       1           235
        3            38
1       1000         34
2       4            30
0       1000         16
        4            16
1       4            11
0       2            11
1       0             4
4       1             4
2       0             3
4       4             3
        5             3
1       3             2
4       3             2
        1000          2
Name: count, dtype: int64

# Export model

In [30]:
#torch.save(model, r'model\model_ht.pth')
import joblib
joblib.dump(scaler, r"model\ht_scaler.pkl")
torch.save(model.state_dict(), r'model\model_ht_state.pth')
